<a href="https://colab.research.google.com/github/niltonsainz/clipping/blob/main/Clipping_FACIAP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# SISTEMA DE CLIPPING LEGISLATIVO - FACIAP
# Versão 1.0 - Câmara dos Deputados e Senado Federal

import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
from datetime import datetime, timedelta
import re
from urllib.parse import urljoin, urlparse

class NewsClippingSystem:
    def __init__(self):
        self.headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
        }
        self.data_limite = datetime(2025, 7, 1)  # 01/07/2025
        self.noticias_coletadas = []
        # Removido sistema de duplicatas conforme solicitado pelo usuário

    def parse_date_camara(self, date_string):
        """Converte data da Câmara (17/07/2025 15:31) para datetime"""
        try:
            return datetime.strptime(date_string.strip(), '%d/%m/%Y %H:%M')
        except ValueError:
            try:
                return datetime.strptime(date_string.strip(), '%d/%m/%Y')
            except ValueError:
                return None

    def parse_date_senado(self, date_string):
        """Converte data do Senado (17/07/2025 15h34) para datetime"""
        try:
            # Remove 'h' e substitui por ':'
            date_clean = date_string.strip().replace('h', ':')
            return datetime.strptime(date_clean, '%d/%m/%Y %H:%M')
        except ValueError:
            try:
                # Tenta só a data
                date_only = date_string.split()[0]
                return datetime.strptime(date_only, '%d/%m/%Y')
            except ValueError:
                return None

    def is_date_valid(self, news_date):
        """Verifica se a data da notícia é posterior a 01/07/2025"""
        if news_date is None:
            return False
        return news_date >= self.data_limite

    def testar_paginacao_senado_corrigida(self):
        """
        Testa a paginação correta do Senado baseada nos prints fornecidos
        """
        print("🔧 TESTANDO PAGINAÇÃO CORRIGIDA DO SENADO...")

        # Testa as primeiras 3 páginas
        for pagina in range(1, 4):
            try:
                if pagina == 1:
                    url = "https://www12.senado.leg.br/noticias/ultimas"
                else:
                    url = f"https://www12.senado.leg.br/noticias/ultimas/{pagina}"

                print(f"\n📄 Testando página {pagina}: {url}")
                response = requests.get(url, headers=self.headers, timeout=30)

                if response.status_code == 200:
                    soup = BeautifulSoup(response.content, 'html.parser')

                    # Busca notícias
                    links = soup.find_all('a', href=lambda x: x and '/noticias/' in x)
                    noticias_encontradas = 0

                    for link in links[:10]:  # Testa apenas os primeiros 10
                        titulo = link.get_text(strip=True)
                        if len(titulo) > 10:  # Só conta títulos reais
                            noticias_encontradas += 1

                    print(f"✅ Status: {response.status_code}")
                    print(f"📰 Notícias válidas encontradas: {noticias_encontradas}")

                    if noticias_encontradas > 0:
                        print(f"🎯 Página {pagina} funcionando corretamente!")
                    else:
                        print(f"⚠️ Página {pagina} sem notícias válidas")

                else:
                    print(f"❌ Status: {response.status_code}")

            except Exception as e:
                print(f"❌ Erro na página {pagina}: {e}")

    def scrape_camara_noticias(self, max_pages=50):
        """
        Scraping das notícias da Câmara dos Deputados
        Coleta desde 01/07/2025 até as mais recentes
        """
        print("🏛️ Iniciando coleta da Câmara dos Deputados...")
        print(f"📅 Buscando notícias desde {self.data_limite.strftime('%d/%m/%Y')} até hoje...")
        url_base = "https://www.camara.leg.br/noticias/ultimas"
        noticias = []
        encontrou_data_limite = False

        for pagina in range(1, max_pages + 1):
            try:
                # Para páginas adicionais, usa o parâmetro de paginação
                url = f"{url_base}?pagina={pagina}" if pagina > 1 else url_base

                print(f"📄 Processando página {pagina}: {url}")
                response = requests.get(url, headers=self.headers, timeout=30)
                response.raise_for_status()

                soup = BeautifulSoup(response.content, 'html.parser')

                # Encontra o container de notícias
                container = soup.find('ul', class_='l-lista-noticias')
                if not container:
                    print(f"⚠️ Container não encontrado na página {pagina}")
                    break

                # Encontra todas as notícias
                articles = container.find_all('article', class_=lambda x: x and 'chamada' in x)

                if not articles:
                    print(f"⚠️ Nenhuma notícia encontrada na página {pagina}")
                    break

                print(f"📰 Encontradas {len(articles)} notícias na página {pagina}")

                noticias_validas_pagina = 0
                noticias_antigas_pagina = 0

                for article in articles:
                    try:
                        # Extrai título e link
                        title_link = article.find('a', class_=lambda x: x and 'titulo' in x)
                        if not title_link:
                            title_link = article.find('a')

                        if title_link:
                            titulo = title_link.get_text(strip=True)
                            link = urljoin(url_base, title_link.get('href', ''))
                        else:
                            continue

                        # Extrai data
                        data_element = article.find('span', class_=lambda x: x and 'data' in x)
                        if data_element:
                            data_texto = data_element.get_text(strip=True)
                            data_noticia = self.parse_date_camara(data_texto)
                        else:
                            continue

                        # Verifica se a data é válida (posterior a 01/07/2025)
                        if self.is_date_valid(data_noticia):
                            noticia = {
                                'fonte': 'Câmara dos Deputados',
                                'titulo': titulo,
                                'link': link,
                                'data_publicacao': data_noticia,
                                'data_texto': data_texto,
                                'data_coleta': datetime.now()
                            }

                            noticias.append(noticia)
                            noticias_validas_pagina += 1
                            print(f"✅ Coletada: {titulo[:50]}... ({data_texto})")

                        # Se encontrou notícia anterior a 01/07, marca flag
                        elif data_noticia and data_noticia < self.data_limite:
                            noticias_antigas_pagina += 1
                            print(f"⏰ Notícia anterior ao limite: {data_texto}")

                    except Exception as e:
                        print(f"❌ Erro ao processar notícia: {e}")
                        continue

                print(f"📊 Página {pagina}: {noticias_validas_pagina} válidas, {noticias_antigas_pagina} anteriores ao limite")

                # Se encontrou muitas notícias antigas (10+), provavelmente chegou no limite
                if noticias_antigas_pagina >= 10:
                    print(f"🎯 Atingiu o limite temporal na página {pagina}. Parando coleta da Câmara.")
                    encontrou_data_limite = True
                    break

                # Se não encontrou nenhuma notícia válida em várias páginas seguidas
                if noticias_validas_pagina == 0 and pagina > 10:
                    print(f"🎯 Nenhuma notícia válida encontrada. Parando coleta da Câmara.")
                    break

                time.sleep(2)  # Pausa entre requisições

            except Exception as e:
                print(f"❌ Erro na página {pagina}: {e}")
                break

        if encontrou_data_limite:
            print(f"✅ Câmara: Coleta completa desde 01/07/2025 - {len(noticias)} notícias coletadas")
        else:
            print(f"✅ Câmara: {len(noticias)} notícias coletadas (pode haver mais páginas)")

        return noticias

    def scrape_senado_noticias(self, max_pages=50):
        """
        Scraping das notícias do Senado Federal
        Coleta desde 01/07/2025 até as mais recentes
        """
        print("🏛️ Iniciando coleta do Senado Federal...")
        print(f"📅 Buscando notícias desde {self.data_limite.strftime('%d/%m/%Y')} até hoje...")
        url_base = "https://www12.senado.leg.br/noticias/ultimas"
        noticias = []
        encontrou_data_limite = False

        for pagina in range(1, max_pages + 1):
            try:
                # URL com paginação correta baseada na imagem do DevTools
                if pagina == 1:
                    url = url_base
                else:
                    # Formato correto confirmado na imagem: /ultimas/2, /ultimas/3, etc.
                    url = f"{url_base}/{pagina}"

                print(f"📄 Processando página {pagina}: {url}")
                response = requests.get(url, headers=self.headers, timeout=30)
                response.raise_for_status()

                soup = BeautifulSoup(response.content, 'html.parser')

                # Múltiplas tentativas para encontrar o container
                container = None

                # Tentativa 1: Container principal que vimos nas imagens
                container = soup.find('div', class_='col-xs-12 col-md-9')

                # Tentativa 2: Lista de resultados
                if not container:
                    container = soup.find('ul', class_='list-unstyled lista-resultados')

                # Tentativa 3: Qualquer div que contenha notícias
                if not container:
                    container = soup.find('div', class_=lambda x: x and 'lista' in x) if soup.find('div', class_=lambda x: x and 'lista' in x) else soup

                if not container:
                    print(f"⚠️ Container não encontrado na página {pagina}")
                    print("🔍 Tentando busca alternativa...")
                    # Busca direta por links de notícias
                    articles = soup.find_all('a', href=lambda x: x and '/noticias/' in x)
                    if articles:
                        print(f"🎯 Encontrados {len(articles)} links alternativos")
                        container = soup  # Usa a página toda como container
                    else:
                        break

                # Estratégia robusta para encontrar notícias
                items = []

                # Tentativa 1: Elementos li (estrutura que esperávamos)
                items = container.find_all('li')

                # Tentativa 2: Se não achou li, busca por divs com classes de notícia
                if not items:
                    items = container.find_all('div', class_=lambda x: x and any(palavra in x for palavra in ['noticia', 'item', 'story', 'article']))

                # Tentativa 3: Busca direta por links de notícias
                if not items:
                    links_noticias = container.find_all('a', href=lambda x: x and '/noticias/' in x)
                    items = [link.parent for link in links_noticias if link.parent]

                if not items:
                    print(f"⚠️ Nenhuma notícia encontrada na página {pagina}")
                    break

                print(f"📰 Encontradas {len(items)} notícias na página {pagina}")

                noticias_validas_pagina = 0
                noticias_antigas_pagina = 0

                for item in items:
                    try:
                        # Busca mais robusta por título e link
                        title_link = None

                        # Tentativa 1: Link direto no item
                        title_link = item.find('a', href=lambda x: x and '/noticias/' in x)

                        # Tentativa 2: Qualquer link no item
                        if not title_link:
                            title_link = item.find('a')

                        if not title_link:
                            continue

                        titulo = title_link.get_text(strip=True)
                        if len(titulo) < 10:  # Título muito curto, provavelmente não é uma notícia
                            continue

                        link = urljoin(url_base, title_link.get('href', ''))

                        # Busca mais robusta por data
                        texto_item = item.get_text()
                        data_match = re.search(r'(\d{2}/\d{2}/\d{4})\s*(\d{2}h\d{2}|\d{2}:\d{2})?', texto_item)

                        if data_match:
                            data_texto = data_match.group(0)
                            data_noticia = self.parse_date_senado(data_texto)
                        else:
                            # Se não achou data no formato esperado, tenta outros formatos
                            data_alt = re.search(r'(\d{2}/\d{2}/\d{4})', texto_item)
                            if data_alt:
                                data_texto = data_alt.group(0)
                                data_noticia = self.parse_date_senado(data_texto)
                            else:
                                continue

                        # Verifica se a data é válida (posterior a 01/07/2025)
                        if self.is_date_valid(data_noticia):
                            noticia = {
                                'fonte': 'Senado Federal',
                                'titulo': titulo,
                                'link': link,
                                'data_publicacao': data_noticia,
                                'data_texto': data_texto,
                                'data_coleta': datetime.now()
                            }

                            noticias.append(noticia)
                            noticias_validas_pagina += 1
                            print(f"✅ Coletada: {titulo[:50]}... ({data_texto})")

                        # Se encontrou notícia anterior a 01/07, marca flag
                        elif data_noticia and data_noticia < self.data_limite:
                            noticias_antigas_pagina += 1
                            print(f"⏰ Notícia anterior ao limite: {data_texto}")

                    except Exception as e:
                        print(f"❌ Erro ao processar notícia: {e}")
                        continue

                print(f"📊 Página {pagina}: {noticias_validas_pagina} válidas, {noticias_antigas_pagina} anteriores ao limite")

                # Se não encontrou nenhuma notícia válida e nem antigas, provavelmente acabaram as notícias
                if noticias_validas_pagina == 0 and noticias_antigas_pagina == 0:
                    print(f"🎯 Página vazia ou sem conteúdo relevante na página {pagina}. Parando coleta do Senado.")
                    break

                # Se encontrou muitas notícias antigas (10+), provavelmente chegou no limite
                if noticias_antigas_pagina >= 10:
                    print(f"🎯 Atingiu o limite temporal na página {pagina}. Parando coleta do Senado.")
                    encontrou_data_limite = True
                    break

                time.sleep(2)  # Pausa entre requisições

            except Exception as e:
                print(f"❌ Erro na página {pagina}: {e}")
                break

        if encontrou_data_limite:
            print(f"✅ Senado: Coleta completa desde 01/07/2025 - {len(noticias)} notícias coletadas")
        else:
            print(f"✅ Senado: {len(noticias)} notícias coletadas (pode haver mais páginas)")

        return noticias

    def debug_senado_structure(self):
        """
        Função de debug para analisar a estrutura da página do Senado
        """
        print("🔍 MODO DEBUG - Analisando estrutura do Senado...")
        url = "https://www12.senado.leg.br/noticias/ultimas"

        try:
            response = requests.get(url, headers=self.headers, timeout=30)
            response.raise_for_status()
            soup = BeautifulSoup(response.content, 'html.parser')

            print(f"✅ Página carregada com sucesso")
            print(f"📄 Título da página: {soup.title.get_text() if soup.title else 'N/A'}")

            # Procura por diferentes containers possíveis
            containers = [
                soup.find('ul', class_='list-unstyled lista-resultados'),
                soup.find('div', class_='col-xs-12 col-md-9'),
                soup.find('div', id='conteudoPrincipal'),
                soup.find('main'),
                soup.find('div', class_=lambda x: x and 'noticias' in x),
            ]

            print("\n🔍 Containers encontrados:")
            for i, container in enumerate(containers):
                if container:
                    print(f"  ✅ Container {i+1}: {container.name} - {container.get('class', [])} - {container.get('id', 'N/A')}")
                else:
                    print(f"  ❌ Container {i+1}: Não encontrado")

            # Busca por todos os links de notícias
            links_noticias = soup.find_all('a', href=lambda x: x and '/noticias/' in x)
            print(f"\n📰 Total de links de notícias encontrados: {len(links_noticias)}")

            if links_noticias:
                print("📋 Primeiros 5 links encontrados:")
                for i, link in enumerate(links_noticias[:5]):
                    titulo = link.get_text(strip=True)
                    href = link.get('href', '')
                    print(f"  {i+1}. {titulo[:60]}... -> {href}")

            # Busca por datas
            texto_pagina = soup.get_text()
            datas_encontradas = re.findall(r'\d{2}/\d{2}/\d{4}', texto_pagina)
            print(f"\n📅 Padrões de data encontrados: {len(set(datas_encontradas))} únicos")
            print(f"    Exemplos: {list(set(datas_encontradas))[:5]}")

            return soup

        except Exception as e:
            print(f"❌ Erro no debug: {e}")
            return None

    def executar_coleta_completa(self, max_pages_camara=50, max_pages_senado=50):
        """
        Executa a coleta completa de ambas as fontes
        Coleta desde 01/07/2025 até as notícias mais recentes
        """
        print("🚀 INICIANDO COLETA HISTÓRICA DE NOTÍCIAS LEGISLATIVAS")
        print("📅 Período: 01/07/2025 até hoje")
        print("=" * 60)

        # Coleta da Câmara
        noticias_camara = self.scrape_camara_noticias(max_pages_camara)

        print("\n" + "=" * 60)

        # Coleta do Senado
        noticias_senado = self.scrape_senado_noticias(max_pages_senado)

        # Consolida resultados
        todas_noticias = noticias_camara + noticias_senado
        self.noticias_coletadas = todas_noticias

        print("\n" + "=" * 60)
        print("📊 RESUMO DA COLETA HISTÓRICA:")
        print(f"🏛️ Câmara dos Deputados: {len(noticias_camara)} notícias")
        print(f"🏛️ Senado Federal: {len(noticias_senado)} notícias")
        print(f"📈 Total coletado: {len(todas_noticias)} notícias")

        if todas_noticias:
            # Calcula estatísticas de período
            datas = [n['data_publicacao'] for n in todas_noticias if n['data_publicacao']]
            if datas:
                data_mais_antiga = min(datas)
                data_mais_recente = max(datas)
                print(f"📅 Período coberto: {data_mais_antiga.strftime('%d/%m/%Y')} a {data_mais_recente.strftime('%d/%m/%Y')}")

        return todas_noticias

    def gerar_dataframe(self):
        """
        Converte as notícias coletadas em DataFrame pandas
        """
        if not self.noticias_coletadas:
            print("⚠️ Nenhuma notícia coletada ainda. Execute primeiro a coleta.")
            return None

        df = pd.DataFrame(self.noticias_coletadas)

        # Ordena por data de publicação (mais recentes primeiro)
        df = df.sort_values('data_publicacao', ascending=False)
        df = df.reset_index(drop=True)

        return df

    def salvar_csv(self, nome_arquivo=None):
        """
        Salva as notícias em arquivo CSV
        """
        df = self.gerar_dataframe()
        if df is None:
            return None

        if nome_arquivo is None:
            timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
            nome_arquivo = f'clipping_legislativo_{timestamp}.csv'

        df.to_csv(nome_arquivo, index=False, encoding='utf-8')
        print(f"💾 Arquivo salvo: {nome_arquivo}")
        return nome_arquivo

# =============================================================================
# EXEMPLO DE USO
# =============================================================================

if __name__ == "__main__":
    # Inicializa o sistema
    clipping = NewsClippingSystem()

    # TESTE RÁPIDO: Descomente para testar a paginação do Senado antes da coleta completa:
    # clipping.testar_paginacao_senado_corrigida()

    # Executa a coleta (ajuste max_pages conforme necessário)
    # Para teste inicial, use números menores (ex: 10, 10)
    # Para coleta completa até 01/07, use números maiores (ex: 50, 50)
    noticias = clipping.executar_coleta_completa(max_pages_camara=50, max_pages_senado=50)

    # Gera DataFrame para análise
    df = clipping.gerar_dataframe()

    if df is not None and len(df) > 0:
        print("\n📋 PREVIEW DOS DADOS:")
        print(df[['fonte', 'titulo', 'data_publicacao']].head(10))

        print(f"\n📊 ESTATÍSTICAS:")
        print(f"Período: {df['data_publicacao'].min()} a {df['data_publicacao'].max()}")
        print(f"Distribuição por fonte:")
        print(df['fonte'].value_counts())

        # Salva em CSV
        arquivo = clipping.salvar_csv()

        print(f"\n✅ Coleta finalizada com sucesso!")
        print(f"📁 Dados disponíveis no DataFrame 'df' e no arquivo '{arquivo}'")

        # Se o Senado não coletou nada, sugere debug
        if 'Senado Federal' not in df['fonte'].values:
            print("\n⚠️ ATENÇÃO: Nenhuma notícia do Senado foi coletada.")
            print("💡 Para testar, execute: clipping.testar_paginacao_senado_corrigida()")

        # Se não chegou até 01/07, sugere aumentar páginas
        if df is not None and len(df) > 0:
            data_mais_antiga = df['data_publicacao'].min()
            if data_mais_antiga > datetime(2025, 7, 1):
                print(f"\n📅 ATENÇÃO: Não chegou até 01/07/2025 (parou em {data_mais_antiga.strftime('%d/%m/%Y')})")
                print("💡 Dica: Aumente o número de páginas para coletar mais histórico.")
                print("💡 Exemplo: clipping.executar_coleta_completa(max_pages_camara=60, max_pages_senado=60)")
    else:
        print("⚠️ Nenhuma notícia foi coletada no período especificado.")

🚀 INICIANDO COLETA HISTÓRICA DE NOTÍCIAS LEGISLATIVAS
📅 Período: 01/07/2025 até hoje
🏛️ Iniciando coleta da Câmara dos Deputados...
📅 Buscando notícias desde 01/07/2025 até hoje...
📄 Processando página 1: https://www.camara.leg.br/noticias/ultimas
📰 Encontradas 10 notícias na página 1
✅ Coletada: Comissão aprova proposta de estímulo fiscal para p... (18/07/2025 18:28)
✅ Coletada: Comissão aprova projeto que obriga serviços de saú... (18/07/2025 18:09)
✅ Coletada: Projeto cria novo marco legal do transporte indivi... (18/07/2025 17:31)
✅ Coletada: Comissão aprova projeto que prevê uso de algoritmo... (18/07/2025 16:57)
✅ Coletada: Hugo Motta e Davi Alcolumbre confirmam recesso par... (18/07/2025 16:29)
✅ Coletada: Oposição acusa Moraes de "abuso de autoridade"; lí... (18/07/2025 16:09)
✅ Coletada: Câmara cria comissão para analisar regulamentação ... (18/07/2025 16:08)
✅ Coletada: Projeto propõe medidas contra assédio e violência ... (18/07/2025 15:23)
✅ Coletada: Proposta amplia limite

In [ ]:
pip install schedule

In [ ]:
import requests
from bs4 import BeautifulSoup
import sqlite3
import pandas as pd
from datetime import datetime
import re
from pathlib import Path

DB_PATH = "clipping_faciap_textos.db"

# Inicializa banco de dados
conn = sqlite3.connect(DB_PATH)
cursor = conn.cursor()

cursor.execute('''
    CREATE TABLE IF NOT EXISTS textos_completos (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        url TEXT UNIQUE,
        fonte TEXT,
        titulo TEXT,
        data_publicacao TEXT,
        texto_completo TEXT,
        num_palavras INTEGER,
        coletado_em TEXT
    )
''')

conn.commit()

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}

def extrair_texto(url):
    try:
        response = requests.get(url, headers=headers, timeout=20)
        soup = BeautifulSoup(response.content, 'html.parser')

        for tag in soup(['script','style','nav','footer','header','aside','video','audio','iframe']):
            tag.decompose()

        texto = None

        if 'camara.leg.br' in url:
            container = soup.find('div', id='content-noticia')
            if container:
                texto = ' '.join([p.get_text(strip=True) for p in container.find_all('p')])

        elif 'senado.leg.br' in url:
            container = (soup.find('div', id='textoMateria') or
                         soup.find('div', class_='texto-materia') or
                         soup.find('div', class_='conteudo-materia'))
            if container:
                texto = ' '.join([p.get_text(strip=True) for p in container.find_all('p')])

        if not texto or len(texto.split()) < 50:
            paragrafos = soup.find_all('p')
            texto = ' '.join([p.get_text(strip=True) for p in paragrafos if len(p.get_text(strip=True)) > 40])

        if texto and len(texto.split()) > 20:
            return texto.strip()
        else:
            return None

    except Exception as e:
        return None

def processar_csv(arquivo):
    df = pd.read_csv(arquivo, encoding='ISO-8859-1', sep=',')
    sucessos = 0
    for _, row in df.iterrows():
        url = row['link']
        if not url:
            continue

        texto = extrair_texto(url)
        if texto:
            conn = sqlite3.connect(DB_PATH)
            cursor = conn.cursor()

            cursor.execute('''
                INSERT OR IGNORE INTO textos_completos (url, fonte, titulo, data_publicacao, texto_completo, num_palavras, coletado_em)
                VALUES (?, ?, ?, ?, ?, ?, ?)
            ''', (
                url,
                row.get('fonte', ''),
                row.get('titulo', ''),
                row.get('data_publicacao', ''),
                texto,
                len(texto.split()),
                datetime.now().isoformat()
            ))
            conn.commit()
            conn.close()
            sucessos += 1
            print(f"✅ Texto extraído ({len(texto.split())} palavras) - {url}")
        else:
            print(f"❌ Falha na extração - {url}")
    print(f"\n🎯 Total de textos coletados com sucesso: {sucessos}")

# Exemplo de uso
if __name__ == '__main__':
    caminho_csv = "clipping_legislativo_20250721_031549.csv"
    if Path(caminho_csv).exists():
        processar_csv(caminho_csv)
    else:
        print("Arquivo CSV não encontrado.")

✅ Texto extraído (556 palavras) - https://www.camara.leg.br/noticias/1182032-comissao-aprova-proposta-de-estimulo-fiscal-para-pesquisa-de-racao-sustentavel/
✅ Texto extraído (602 palavras) - https://www.camara.leg.br/noticias/1182017-comissao-aprova-projeto-que-obriga-servicos-de-saude-a-assegurar-mamografia-a-mulheres-com-deficiencia/
✅ Texto extraído (179 palavras) - https://www12.senado.leg.br/noticias/materias/2025/07/18/sancionada-homenagem-a-jose-pereira-da-silva-em-viaduto-em-pouso-alegre
✅ Texto extraído (1428 palavras) - https://www.camara.leg.br/noticias/1181948-projeto-cria-novo-marco-legal-do-transporte-individual-por-aplicativo-no-pais/
✅ Texto extraído (1397 palavras) - https://www12.senado.leg.br/noticias/materias/2025/07/18/senado-deve-analisar-fundo-social-do-pre-sal-para-agricultores-em-calamidades
✅ Texto extraído (898 palavras) - https://www.camara.leg.br/noticias/1182008-comissao-aprova-projeto-que-preve-uso-de-algoritmos-no-combate-a-crimes-digitais-contra-crianca

In [ ]:
import sqlite3
import pandas as pd

conn = sqlite3.connect("clipping_faciap_textos.db")
df = pd.read_sql_query("SELECT * FROM textos_completos", conn)
conn.close()

df.head()

,id,url,fonte,titulo,data_publicacao,texto_completo,num_palavras,coletado_em
0,1,https://www.camara.leg.br/noticias/1182032-com...,CÃ¢mara dos Deputados,ComissÃ£o aprova proposta de estÃ­mulo fiscal ...,2025-07-18 18:28:00,Projeto será analisado por outras três comissõ...,556,2025-07-21T05:01:54.805323
1,2,https://www.camara.leg.br/noticias/1182017-com...,CÃ¢mara dos Deputados,ComissÃ£o aprova projeto que obriga serviÃ§os ...,2025-07-18 18:09:00,Vinicius Loures/Câmara dos DeputadosDeputado T...,602,2025-07-21T05:01:55.394872
2,3,https://www12.senado.leg.br/noticias/materias/...,Senado Federal,Sancionada homenagem a JosÃ© Pereira da Silva ...,2025-07-18 17:38:00,Passa a se chamar Deputado José Pereira da Sil...,179,2025-07-21T05:01:56.367900
3,4,https://www.camara.leg.br/noticias/1181948-pro...,CÃ¢mara dos Deputados,Projeto cria novo marco legal do transporte in...,2025-07-18 17:31:00,Kayo Magalhães / Câmara dos DeputadosLuiz Gast...,1428,2025-07-21T05:01:58.645875
4,5,https://www12.senado.leg.br/noticias/materias/...,Senado Federal,Senado deve analisar Fundo Social do prÃ©-sal ...,2025-07-18 17:16:00,O Senado deve examinar no segundo semestre o p...,1397,2025-07-21T05:01:59.510614


In [ ]:
!python -m spacy download pt_core_news_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 27.4 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('pt_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
import sqlite3
import pandas as pd
import spacy
import unicodedata
import re
from datetime import datetime

# Carrega modelo spaCy para lematização em português
nlp = spacy.load("pt_core_news_sm")

# Funções de pré-processamento

def limpar_texto(texto):
    texto = texto.lower()
    texto = unicodedata.normalize('NFKD', texto).encode('ASCII', 'ignore').decode('utf-8')
    texto = re.sub(r'[\W_]+', ' ', texto)  # Remove pontuação
    return texto.strip()

def lematizar_texto(texto):
    doc = nlp(texto)
    return ' '.join([token.lemma_ for token in doc if not token.is_stop])

# Carrega dicionário de palavras-chave
df_dict = pd.read_csv("dicionario_faciap.csv", sep=';')
df_dict['palavra_chave'] = df_dict['palavra_chave'].str.lower()

# Converte vírgulas para pontos nas colunas numéricas
df_dict['peso_risco'] = df_dict['peso_risco'].astype(str).str.replace(',', '.').astype(float)
df_dict['peso_interesse'] = df_dict['peso_interesse'].astype(str).str.replace(',', '.').astype(float)

# Conecta ao banco de dados
conn = sqlite3.connect("clipping_faciap_textos.db")
df_noticias = pd.read_sql_query("SELECT * FROM textos_completos", conn)

resultados = []

# Calcula o score para cada notícia
for _, row in df_noticias.iterrows():
    texto_bruto = row['texto_completo']
    texto = limpar_texto(texto_bruto)
    texto_lema = lematizar_texto(texto)

    score_risco = 0
    score_interesse = 0
    palavras_ativadas = []
    eixo_dominante = None

    for _, item in df_dict.iterrows():
        termo = item['palavra_chave']
        ocorrencias = texto_lema.count(termo.lower())
        if ocorrencias > 0:
            score_risco += ocorrencias * float(item['peso_risco'])
            score_interesse += ocorrencias * float(item['peso_interesse'])
            palavras_ativadas.append(termo)

    if palavras_ativadas:
        # Eixo mais frequente entre as palavras ativadas
        eixos_filtrados = df_dict[df_dict['palavra_chave'].isin(palavras_ativadas)]['eixo_temat']
        eixo_dominante = eixos_filtrados.mode().iloc[0] if not eixos_filtrados.empty else None

    resultados.append({
        'noticia_id': row['id'],
        'score_risco': score_risco,
        'score_interesse': score_interesse,
        'palavras_chave': ', '.join(set(palavras_ativadas)),
        'categoria': eixo_dominante,
        'data_avaliacao': datetime.now().isoformat()
    })

# Cria nova tabela de scores e salva
df_resultados = pd.DataFrame(resultados)
df_resultados.to_sql("scores_faciap", conn, if_exists='replace', index=False)
conn.commit()
conn.close()

print("✅ Cálculo de Score FACIAP concluído e salvo no banco de dados.")

✅ Cálculo de Score FACIAP concluído e salvo no banco de dados.


In [ ]:
import sqlite3
import pandas as pd

conn = sqlite3.connect("clipping_faciap_textos.db")
df_scores = pd.read_sql_query("SELECT * FROM scores_faciap", conn)
conn.close()

df_scores.to_csv("scores_faciap_resultados.csv", index=False, encoding='utf-8')

In [ ]:
import sqlite3
conn = sqlite3.connect("clipping_faciap_textos.db")
cursor = conn.cursor()

# Lista todas as tabelas
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tabelas = cursor.fetchall()
print("Tabelas:", tabelas)

# Para cada tabela, mostra a estrutura
for tabela in tabelas:
    nome_tabela = tabela[0]
    cursor.execute(f"PRAGMA table_info({nome_tabela})")
    colunas = cursor.fetchall()
    print(f"\nEstrutura da tabela '{nome_tabela}':")
    for coluna in colunas:
        print(f"  {coluna[1]} ({coluna[2]})")

conn.close()

Tabelas: [('textos_completos',), ('sqlite_sequence',), ('scores_faciap',)]

Estrutura da tabela 'textos_completos':
  id (INTEGER)
  url (TEXT)
  fonte (TEXT)
  titulo (TEXT)
  data_publicacao (TEXT)
  texto_completo (TEXT)
  num_palavras (INTEGER)
  coletado_em (TEXT)

Estrutura da tabela 'sqlite_sequence':
  name ()
  seq ()

Estrutura da tabela 'scores_faciap':
  noticia_id (INTEGER)
  score_risco (REAL)
  score_interesse (REAL)
  palavras_chave (TEXT)
  categoria (TEXT)
  data_avaliacao (TEXT)


In [ ]:
import pandas as pd
import sqlite3

conn = sqlite3.connect("clipping_faciap_textos.db")
df_amostra = pd.read_sql_query("SELECT * FROM textos_completos LIMIT 3", conn)
print(df_amostra.columns.tolist())
print(df_amostra.head())
conn.close()

['id', 'url', 'fonte', 'titulo', 'data_publicacao', 'texto_completo', 'num_palavras', 'coletado_em']
   id                                                url  \
0   1  https://www.camara.leg.br/noticias/1182032-com...   
1   2  https://www.camara.leg.br/noticias/1182017-com...   
2   3  https://www12.senado.leg.br/noticias/materias/...   

                   fonte                                             titulo  \
0  CÃ¢mara dos Deputados  ComissÃ£o aprova proposta de estÃ­mulo fiscal ...   
1  CÃ¢mara dos Deputados  ComissÃ£o aprova projeto que obriga serviÃ§os ...   
2         Senado Federal  Sancionada homenagem a JosÃ© Pereira da Silva ...   

       data_publicacao                                     texto_completo  \
0  2025-07-18 18:28:00  Projeto será analisado por outras três comissõ...   
1  2025-07-18 18:09:00  Vinicius Loures/Câmara dos DeputadosDeputado T...   
2  2025-07-18 17:38:00  Passa a se chamar Deputado José Pereira da Sil...   

   num_palavras                 